In [58]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [59]:
#Get relative path of the CSV File 
#Enumerate therough all the files in the folder and check if the path exists

mypath = os.path.dirname(os.path.realpath('train.csv'))
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
os.path.exists(mypath)

True

In [61]:
print(onlyfiles[21])

train.csv


In [63]:
#Read Training CSV for house regression
dataset = pd.read_csv(onlyfiles[21])
dataset.shape

(1460, 81)

importing the dataset 
encode categorical data
Backward elimination

In [64]:
#Delete redundant features with over 50% NAN values E.G: porch sizes, pool area, alley, miscallenous features and its value 
unwantedFeaturesList = ["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea",
                       "PoolQC","Fence","Alley","MiscFeature","MiscVal"]
dataset.drop(unwantedFeaturesList,inplace=True,axis=1)

In [65]:
# Check all the columns fetures whcih has a numerical value type
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))
is_number(dataset.dtypes)

array([ True,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True, False,
       False, False,  True,  True, False, False,  True], dtype=bool)

In [66]:
#Separate and select dataframes by dataTypes 
df_NonNumericalType = dataset.select_dtypes(exclude=[np.number])
df_NumericalType = dataset.select_dtypes(include=[np.number])

#Set type to categorical
df_NonNumericalType[df_NonNumericalType.select_dtypes(['object']).columns] = df_NonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df_NonNumericalType.dtypes

C:\Users\BOBBOO\Anaconda3\lib\site-packages\pandas\core\frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


MSZoning         category
Street           category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
ExterQual        category
ExterCond        category
Foundation       category
BsmtQual         category
BsmtCond         category
BsmtExposure     category
BsmtFinType1     category
BsmtFinType2     category
Heating          category
HeatingQC        category
CentralAir       category
Electrical       category
KitchenQual      category
Functional       category
FireplaceQu      category
GarageType       category
GarageFinish     category
GarageQual       category
GarageCond       category
PavedDrive       category
SaleType         category
SaleConditio

In [69]:
#replace all the NA values to numpy NAN for float 64 data type
df_NumericalType.replace('NA',np.nan)
#Select Numerical values except the index 
xNumerical = df_NumericalType.values
#drop the SalesPrice column
df_NumericalType.drop(["SalePrice"], inplace=True, axis=1)

C:\Users\BOBBOO\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [70]:
#Handle missing data for numerical values 
from sklearn.preprocessing import Imputer
if df_NumericalType.isnull().values.any() :
    #mean is the deafualt parameter anywyas
    imputer = Imputer(missing_values = 'NaN', strategy='mean', axis=0)
    imputer.fit(xNumerical[:,2:33])
    xNumerical[:,2:33] = imputer.transform(xNumerical[:,2:33])

In [74]:
#Scaling Numerical data
from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
NumericalScaledData = sc_X.fit_transform(xNumerical)


dtype('float64')

In [72]:
#Sort by categorical data and encoding it
#Create dummy data for all the categorical columns
cat_features = [key for key in dict(df_NonNumericalType.dtypes) if dict(df_NonNumericalType.dtypes)[key] in ['category']]
df_NonNumericalencoded = pd.get_dummies(df_NonNumericalType, columns=cat_features, drop_first=True)
df_NonNumericalencoded.reset_index(level=0, inplace=True)
df_NumericalType.reset_index(level=0, inplace=True)

In [43]:
#Drop column ID from numericalType
df_NumericalType.drop('Id', axis=1)

,index,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,MoSold,YrSold,SalePrice
0,0,60,65.0,8450,7,5,2003,2003,196.0,706,...,3,1,8,0,2003.0,2,548,2,2008,208500
1,1,20,80.0,9600,6,8,1976,1976,0.0,978,...,3,1,6,1,1976.0,2,460,5,2007,181500
2,2,60,68.0,11250,7,5,2001,2002,162.0,486,...,3,1,6,1,2001.0,2,608,9,2008,223500
3,3,70,60.0,9550,7,5,1915,1970,0.0,216,...,3,1,7,1,1998.0,3,642,2,2006,140000
4,4,60,84.0,14260,8,5,2000,2000,350.0,655,...,4,1,9,1,2000.0,3,836,12,2008,250000
5,5,50,85.0,14115,5,5,1993,1995,0.0,732,...,1,1,5,0,1993.0,2,480,10,2009,143000
6,6,20,75.0,10084,8,5,2004,2005,186.0,1369,...,3,1,7,1,2004.0,2,636,8,2007,307000
7,7,60,NaN,10382,7,6,1973,1973,240.0,859,...,3,1,7,2,1973.0,2,484,11,2009,200000
8,8,50,51.0,6120,7,5,1931,1950,0.0,0,...,2,2,8,2,1931.0,2,468,4,2008,129900
9,9,190,50.0,7420,5,6,1939,1950,0.0,851,...,2,2,5,2,1939.0,1,205,1,2008,118000


In [84]:
df_NumericalScaled = pd.DataFrame(NumericalScaledData)
df_NumericalScaled.columns = df_NumericalType.columns
df_AllTypes = pd.concat([df_NumericalScaled, df_NonNumericalencoded], axis=1)

In [85]:
df_AllTypes.columns

Index(['index', 'Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       ...
       'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
       'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=232)

In [120]:
#x = dataset.iloc[:,:-1].values
#y = dataset.iloc[:,80].values